In [ ]:
import base64
import hashlib
from PIL import Image, UnidentifiedImageError
from os import system, name
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.backends import default_backend
from cryptography.exceptions import InvalidSignature

# Função para embutir texto em uma imagem usando esteganografia
def embed_text(image_path, message, output_path="new_image.png"):
    try:
        image = Image.open(image_path)
        encoded = image.copy()

        width, height = image.size
        index = 0

        message += '####'  # Delimitador para o fim da mensagem
        binary_message = ''.join([format(ord(i), '08b') for i in message])

        for row in range(height):
            for col in range(width):
                if index < len(binary_message):
                    pixel = list(encoded.getpixel((col, row)))
                    for n in range(3):
                        if index < len(binary_message):
                            pixel[n] = int(format(pixel[n], '08b')[:-1] + binary_message[index], 2)
                            index += 1
                    encoded.putpixel((col, row), tuple(pixel))
        encoded.save(output_path)
        print(f"Mensagem embutida com sucesso em {output_path}")
    except FileNotFoundError:
        print("Erro: Arquivo de imagem não encontrado.")
    except UnidentifiedImageError:
        print("Erro: O arquivo selecionado não é uma imagem válida.")

# Função para recuperar texto de uma imagem
def retrieve_text(image_path):
    try:
        image = Image.open(image_path)
        binary_data = ""

        for row in range(image.size[1]):
            for col in range(image.size[0]):
                pixel = image.getpixel((col, row))
                for n in range(3):
                    binary_data += format(pixel[n], '08b')[-1]

        all_bytes = [binary_data[i:i+8] for i in range(0, len(binary_data), 8)]
        decoded_message = ""

        for byte in all_bytes:
            decoded_message += chr(int(byte, 2))
            if decoded_message[-4:] == "####":
                break

        return decoded_message[:-4]
    except FileNotFoundError:
        print("Erro: Arquivo de imagem não encontrado.")
    except UnidentifiedImageError:
        print("Erro: O arquivo selecionado não é uma imagem válida.")
        return None

# Função para gerar hash de uma imagem
def generate_hash(image_path):
    try:
        with open(image_path, "rb") as img_file:
            img_data = img_file.read()
            return hashlib.sha256(img_data).hexdigest()
    except FileNotFoundError:
        print("Erro: Arquivo de imagem não encontrado.")
        return None

# Funções de criptografia
def generate_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    return private_key, public_key

def encrypt_message(public_key, message):
    try:
        encrypted = public_key.encrypt(
            message.encode(),
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        return encrypted
    except TypeError:
        print("Erro: A mensagem para encriptação deve ser uma string válida.")
        return None

def decrypt_message(private_key, encrypted_message):
    try:
        decrypted = private_key.decrypt(
            encrypted_message,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        return decrypted.decode()
    except (ValueError, InvalidSignature):
        print("Erro: Falha na decriptação. A mensagem pode estar corrompida ou a chave está incorreta.")
        return None

# Função do Menu Principal
def menu():
    private_key, public_key = generate_keys()

    while True:
        print("\nMenu de opções:")
        print("(1) Embutir texto em imagem (Steganography)")
        print("(2) Recuperar texto de imagem (Steganography)")
        print("(3) Gerar hash das imagens")
        print("(4) Encriptar mensagem")
        print("(5) Decriptar mensagem")
        print("(S ou s) Sair")

        option = input("Escolha uma opção: ")

        if option == '1':
            image_path = input("Digite o caminho da imagem: ")
            message = input("Digite a mensagem para embutir: ")
            embed_text(image_path, message)

        elif option == '2':
            image_path = input("Digite o caminho da imagem alterada: ")
            retrieved_message = retrieve_text(image_path)
            if retrieved_message:
                print("Mensagem recuperada:", retrieved_message)

        elif option == '3':
            original_image = input("Digite o caminho da imagem original: ")
            altered_image = input("Digite o caminho da imagem alterada: ")
            original_hash = generate_hash(original_image)
            altered_hash = generate_hash(altered_image)
            if original_hash:
                print("Hash da imagem original:", original_hash)
            if altered_hash:
                print("Hash da imagem alterada:", altered_hash)

        elif option == '4':
            message = input("Digite a mensagem para encriptar: ")
            encrypted_message = encrypt_message(public_key, message)
            if encrypted_message:
                print("Mensagem encriptada:", base64.b64encode(encrypted_message).decode())
                save_option = input("Deseja embutir a mensagem encriptada em uma imagem? (S/N): ")
                if save_option.lower() == 's':
                    image_path = input("Digite o caminho da imagem: ")
                    embed_text(image_path, base64.b64encode(encrypted_message).decode())

        elif option == '5':
            image_path = input("Digite o caminho da imagem com mensagem encriptada: ")
            encrypted_message = retrieve_text(image_path)
            if encrypted_message:
                decrypted_message = decrypt_message(private_key, base64.b64decode(encrypted_message))
                if decrypted_message:
                    print("Mensagem decriptada:", decrypted_message)

        elif option.lower() == 's':
            print("Encerrando aplicação...")
            break

        else:
            print("Opção inválida. Tente novamente.")
        if name == "nt":
          system("cls")
        else:
          system("clear")

menu()



Menu de opções:
(1) Embutir texto em imagem (Steganography)
(2) Recuperar texto de imagem (Steganography)
(3) Gerar hash das imagens
(4) Encriptar mensagem
(5) Decriptar mensagem
(S ou s) Sair
Escolha uma opção: 1
Digite o caminho da imagem: third_busta.png
Digite a mensagem para embutir: Mensagem a ser criptografada kkk loucura
Mensagem embutida com sucesso em new_image.png

Menu de opções:
(1) Embutir texto em imagem (Steganography)
(2) Recuperar texto de imagem (Steganography)
(3) Gerar hash das imagens
(4) Encriptar mensagem
(5) Decriptar mensagem
(S ou s) Sair
Escolha uma opção: 2
